In [16]:
try:
    import pyNN.spiNNaker as sim
except Exception:
    import spynnaker8 as sim
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

In [17]:
#spynnaker setup
sim.setup(timestep=0.1, min_delay=0.1, max_delay=1.0)

2020-04-28 20:53:05 INFO: Read cfg files: /home/jovyan/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/jovyan/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/jovyan/.spynnaker.cfg
2020-04-28 20:53:05 INFO: Will search these locations for binaries: /home/jovyan/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries : /home/jovyan/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries
2020-04-28 20:53:05 WARNING: /home/jovyan/SpeechRecognition/reports has 13 old reports that have not been closed
2020-04-28 20:53:05 WARNING: /home/jovyan/SpeechRecognition/application_generated_data_files has 13 old reports that have not been closed
2020-04-28 20:53:05 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-04-28 20:53:05 INFO: Setting time sca

['/home/jovyan/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/jovyan/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/jovyan/.spynnaker.cfg']


0

In [18]:
model = sim.extra_models.Izhikevich_cond
# param_a = 0.03
# param_b = -2
# param_c = -50
# param_d = 100
# param_u = 0
# param_v = -60

# cell_params =  {'a': param_a, 'b': param_b, 'c': param_c, 'd': param_d, 'v': param_v, 
#                 'u': param_u, 'tau_syn_E': 6, 'tau_syn_I': 4, 'i_offset':5}

snr_a=0.02 #0.005
snr_b=0.2 #0.32
snr_c=-65 #-65
snr_d=8 #2
snr_v_init = 30 #-70
snr_u_init = -13 #snr_b * snr_v_init

weight_value_plastic = 0.001
weight_value_static = 0.1

tau_ampa = 6###excitatory synapse time constant
tau_gabaa= 4### inhibitory synapse time constant
E_ampa = 0.0
E_gabaa = -80.0
current_bias = 4.0
cell_params = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                   'v': snr_v_init, 'u': snr_u_init,
                   'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                   'i_offset': current_bias,
                   'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                   }

e_rate = 80
in_rate = 300

n_stim_test = 5
n_stim_pairing = 20
dur_stim = 20

pop_size = 40

ISI = 90
start_test_pre_pairing = 200
start_pairing = 1500
start_test_post_pairing = 700

simtime = (start_pairing + start_test_post_pairing + ISI*(n_stim_pairing+n_stim_test) + 550.)

IAddPre = []
IAddPost = []

pre_pop = sim.Population(pop_size, model(**cell_params))
post_pop = sim.Population(pop_size, model(**cell_params))

for i in range(n_stim_test):
    IAddPre.append(sim.Population(pop_size, sim.SpikeSourcePoisson(rate=in_rate, start = start_test_pre_pairing + ISI * i, duration = dur_stim)))

for i in range(n_stim_pairing):
    IAddPre.append(sim.Population(pop_size, sim.SpikeSourcePoisson(rate = in_rate, start = start_pairing  + ISI * i, duration = dur_stim)))
    IAddPost.append(sim.Population(pop_size, sim.SpikeSourcePoisson(rate = in_rate, start = start_pairing  + ISI * i + 10., duration = dur_stim)))
    
for i in range(n_stim_test):
    IAddPre.append(sim.Population(pop_size, sim.SpikeSourcePoisson(rate = in_rate, start = start_pairing  + ISI * n_stim_pairing + start_test_post_pairing + ISI * i, duration = dur_stim)))
    
INoisePre = sim.Population(pop_size, sim.SpikeSourcePoisson(rate = e_rate, start = 0, duration = simtime), label="expoisson")
INoisePost = sim.Population(pop_size, sim.SpikeSourcePoisson(rate = e_rate, start = 0, duration = simtime), label="expoisson")

JEE = 3
ee_connector = sim.OneToOneConnector()

sim.Projection(INoisePre, pre_pop, ee_connector, receptor_type = 'excitatory', synapse_type = sim.StaticSynapse(weight = weight_value_static))
sim.Projection(INoisePost, post_pop, ee_connector, receptor_type = 'excitatory', synapse_type = sim.StaticSynapse(weight = weight_value_static))

for i in range(len(IAddPre)):
    sim.Projection(IAddPre[i], pre_pop, ee_connector, receptor_type = 'excitatory', synapse_type = sim.StaticSynapse(weight = weight_value_static))
    
for i in range(len(IAddPost)):
    sim.Projection(IAddPost[i], post_pop, ee_connector, receptor_type = 'excitatory', synapse_type = sim.StaticSynapse(weight = weight_value_static))
    
stdp_model = sim.STDPMechanism(timing_dependence=sim.SpikePairRuleAntiHebbian(tau_plus = 20., tau_minus = 20., A_plus = 0.02, A_minus = 0.02), weight_dependence = sim.MultiplicativeWeightDependence(w_min=0.001, w_max=0.01), weight=weight_value_plastic)

plastic_projection = sim.Projection(pre_pop, post_pop, sim.FixedProbabilityConnector(p_connect=0.5), synapse_type=stdp_model)

#simulation and results
pre_pop.record(['v', 'spikes'])
post_pop.record(['v', 'spikes'])

sim.run(simtime)

print("Weights:{}".format(plastic_projection.get('weight', 'list')))

pre_spikes = pre_pop.get_data('spikes')
post_spikes = post_pop.get_data('spikes')

Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,simtime)), Panel(post_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,simtime)), title="stdp example", annotations="simulated with {}".format(sim.name())).save('MultiAntiHebbianTesting(0.001to0.01)(weights:0.001,0.1).png')


sim.end()


    
    

2020-04-28 20:53:06 WARNING: Formal PyNN specifies that v should be set using initial_values not cell_params
2020-04-28 20:53:06 WARNING: Formal PyNN specifies that u should be set using initial_values not cell_params
2020-04-28 20:53:06 WARNING: Formal PyNN specifies that v should be set using initial_values not cell_params
2020-04-28 20:53:06 WARNING: Formal PyNN specifies that u should be set using initial_values not cell_params
2020-04-28 20:53:06 INFO: Starting execution process
2020-04-28 20:53:06 INFO: Simulating for 50000 0.1ms timesteps using a hardware timestep of 10us
2020-04-28 20:53:10 INFO: Time 0:00:03.826938 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-04-28 20:53:19 INFO: Time 0:00:09.410853 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph e

Weights:[( 0,  2, 13.01025391) ( 0,  3,  7.60620117) ( 0,  4, 10.54174805)
 ( 0,  6,  9.37109375) ( 0,  7, 13.171875  ) ( 0,  8,  5.2487793 )
 ( 0, 13,  0.53417969) ( 0, 15, 12.0402832 ) ( 0, 19,  5.14208984)
 ( 0, 20,  0.95874023) ( 0, 21, 10.00219727) ( 0, 22, 10.82104492)
 ( 0, 23, 11.87963867) ( 0, 25,  0.78930664) ( 0, 26,  8.65380859)
 ( 0, 27,  7.73291016) ( 0, 29, 12.12573242) ( 0, 33, 11.60864258)
 ( 0, 34,  7.89819336) ( 0, 35,  4.9675293 ) ( 0, 37,  6.92089844)
 ( 1,  7, 11.05883789) ( 1,  8,  7.42236328) ( 1,  9,  4.36816406)
 ( 1, 10,  2.55786133) ( 1, 11,  6.10327148) ( 1, 13,  4.34399414)
 ( 1, 16, 10.12646484) ( 1, 18,  5.55444336) ( 1, 19,  7.00463867)
 ( 1, 20,  4.31469727) ( 1, 22,  8.60693359) ( 1, 26,  4.13378906)
 ( 1, 27,  9.23876953) ( 1, 28,  4.68676758) ( 1, 29,  5.01000977)
 ( 1, 31, 11.04638672) ( 1, 32,  3.57788086) ( 1, 33,  9.36914062)
 ( 1, 35, 12.90795898) ( 1, 36, 14.24560547) ( 1, 37, 13.02148438)
 ( 1, 38, 15.2253418 ) ( 2,  0, 15.27807617) ( 2,  1, 

2020-04-28 20:55:03 WARNING: The weights from the plastic synapses for AbstractPopulationVertex_2:0:39 on 0, 0, 4 saturated 2295 times. If this causes issue increase the spikes_per_second and / or ring_buffer_sigma values located within the .spynnaker.cfg file.


5000.0
